# Featurize a dataset

Any machine learning model will expect tensorial representations of the chemical data. This notebooks provides a workflow to achieve such goal.

`kinoml.dataset.DatasetProvider` objects need to be available to deal with your collection of raw measurements for protein:ligand systems. These objects are, roughly, a list of `kinoml.core.BaseMeasurement`, each containing a set of `.values` and a some extra metadata, like the `system` objects to be featurized here.

In ligand-based models, protein information is only considered marginally, and most of the action happens at the ligand level. Usually starting with a string representation such as SMILES, or a database identifier such as a PubChem ID, these are promoted to (usually) RDKit objects and then transformed into a tensor of some form (e.g. fingerprints, molecular graph as an adjacency matrix, etc).

Available featurizers can be found under `kinoml.features`.

## How to use

Create a copy of this template into a new subdirectory under `features/`. Since the tensor files are not pushed to GitHub for size reasons, they will stay in your machine. Use a unique name for that subdirectory so people can identify creators, preferrably your GitHub username. Each subdirectory should ONLY contain a single notebook and featurization exercise. Then, define the hyperparameters below.

In [1]:
# If this is the template file (and not a copy) and you are introducing changes,
# update VERSION with the current date (YYYY.MM.DD)
VERSION = "2020.12.05" 

## ✏ Define hyper parameters

In [2]:
# TEMPLATE VALUES -- these are overriden (see below if executed) by papermill using a YAML or Python file as input
DATASET_CLS = "import.path.to.DatasetProvider"
DATASET_KWARGS = {"option": "value", "option2": "value2"}

PIPELINE = {
    "someuniquekey": [
        ("import.path.to.SomeFeaturizer", {"option": "value", "option2": "value2"}),
        ("import.path.to.SomeOtherFeaturizer", {"option": "value", "option2": "value2"}),
    ]
}
PIPELINE_AGG = "kinoml.features.core.Concatenated"
PIPELINE_AGG_KWARGS = {}

FEATURIZE_KWARGS = {"processes": 1}

GROUPS = [
    ("kinoml.datasets.groups.CallableGrouper", {"function": "lambda something: something.attribute"}),
    ("kinoml.datasets.groups.CallableGrouper", {"function": "lambda otherthing: otherthing.attribute2"})
]

TRAIN_TEST_VAL_KWARGS = {"idx_train": 0.8, "idx_test": 0.1, "idx_val": 0.1}

## IGNORE THIS ONE
from pathlib import Path
HERE = Path(_dh[-1])

In [3]:
# Parameters
DATASET_CLS = "kinoml.datasets.kinomescan.pkis2.PKIS2DatasetProvider"
DATASET_KWARGS = {}
PIPELINE = {
    "ligand": [
        ["kinoml.features.ligand.SmilesToLigandFeaturizer", {"style": "rdkit"}],
        [
            "kinoml.features.ligand.MorganFingerprintFeaturizer",
            {"nbits": 512, "radius": 2},
        ],
    ]
}
PIPELINE_AGG = "kinoml.features.core.Concatenated"
PIPELINE_AGG_KWARGS = {}
FEATURIZE_KWARGS = {"processes": 1}
GROUPS = [
    [
        "kinoml.datasets.groups.CallableGrouper",
        {"function": "lambda measurement: measurement.system.protein.name"},
    ],
    [
        "kinoml.datasets.groups.CallableGrouper",
        {"function": "lambda measurement: type(measurement).__name__"},
    ],
]
TRAIN_TEST_VAL_KWARGS = {"idx_train": 0.8, "idx_test": 0.1, "idx_val": 0.1}
HERE = (
    "/home/jaime/devel/py/openkinome/experiments-binding-affinity/features/001_morgan"
)


⚠ From here on, you should _not_ need to modify anything else 🤞

---

Define key paths for data and outputs:

In [4]:
HERE = Path(HERE)
for parent in HERE.parents:
    if next(parent.glob(".github/"), None):
        REPO = parent
        break

# Generate paths for this pipeline
featurizer_path = []
for name, branch in PIPELINE.items():
    featurizer_path.append(name)
    for clsname, kwargs in branch:
        clsname = clsname.rsplit(".", 1)[1]
        kwargs = [f"{k}={''.join(c for c in str(v) if c.isalnum())}" for k,v in kwargs.items()]
        featurizer_path.append("_".join([clsname] + kwargs))

OUT = HERE / "_output"  / "__".join(featurizer_path) / DATASET_CLS.rsplit('.', 1)[1]
OUT.mkdir(parents=True, exist_ok=True)

print(f"This notebook:           HERE = ~/{HERE.relative_to(Path.home())}")
print(f"This repo:               REPO = ~/{REPO.relative_to(Path.home())}")
print(f"Outputs in:               OUT = ~/{OUT.relative_to(Path.home())}")

This notebook:           HERE = ~/devel/py/openkinome/experiments-binding-affinity/features/001_morgan
This repo:               REPO = ~/devel/py/openkinome/experiments-binding-affinity
Outputs in:               OUT = ~/devel/py/openkinome/experiments-binding-affinity/features/001_morgan/_output/ligand__SmilesToLigandFeaturizer_style=rdkit__MorganFingerprintFeaturizer_nbits=512_radius=2/PKIS2DatasetProvider


In [5]:
# Nasty trick: save all-caps local variables (CONSTANTS working as hyperparametrs) so far in a dict to save it later
_hparams = {key: value for key, value in locals().items() if key.upper() == key and not key.startswith(("_", "OE_"))}

In [6]:
# Filter out some warnings thrown by openforcefield and rdkit
import warnings
warnings.simplefilter("ignore")
import logging
logging.basicConfig(level=logging.ERROR)
import os
import sys
from pathlib import Path

import numpy as np

from kinoml.utils import seed_everything, import_object
seed_everything();

## Load raw data

In [7]:
dataset = import_object(DATASET_CLS).from_source(**DATASET_KWARGS)[:10]
dataset

<PKIS2DatasetProvider with 10 PercentageDisplacementMeasurement measurements and 9 systems (AminoAcidSequence=9, SmilesLigand=1)>

In [8]:
df = dataset.to_dataframe()
df

,Systems,n_components,PercentageDisplacementMeasurement
0,AAK1 & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)C2...,2,14.0
1,ABL1-nonphosphorylated & Clc1cccc(Cn2c(nn3c2nc...,2,28.0
2,ABL1-nonphosphorylated & Clc1cccc(Cn2c(nn3c2nc...,2,20.0
3,ABL2 & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)C2...,2,5.0
4,ACVR1 & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)C...,2,0.0
5,ACVR1B & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)...,2,0.0
6,ACVR2A & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)...,2,0.0
7,ACVR2B & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)...,2,0.0
8,ACVRL1 & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)...,2,9.0
9,ADCK3 & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)C...,2,11.0


## Featurize

In [9]:
# build pipeline
from kinoml.features.core import Pipeline

featurizers = []
for key, featurizer_instructions in PIPELINE.items():
    featurizers.append(Pipeline([import_object(import_str)(**kwargs) for import_str, kwargs in featurizer_instructions]))
featurizer = import_object(PIPELINE_AGG)(featurizers, **PIPELINE_AGG_KWARGS)

In [10]:
# prefeaturize everything
dataset.featurize(featurizer, **FEATURIZE_KWARGS);

## Filter

Remove systems that couldn't be featurized. Successful featurizations are stored in `measurement.system.featurizations['last']` so we test for that key existence.

In [11]:
from kinoml.datasets.groups import CallableGrouper, RandomGrouper
grouper = CallableGrouper(lambda measurement: 'invalid' if 'last' not in measurement.system.featurizations else 'valid')
grouper.assign(dataset, overwrite=True, progress=False)
groups = dataset.split_by_groups()
if "invalid" in groups:
    _invalid = groups.pop("invalid")
    print(len(_invalid.measurements), "entries could not be featurized!", file=sys.stderr)
    print(_invalid.measurements[0].system.featurizations)

## Groups

Cumulatively apply groups.

In [12]:
groups[("valid",)] = groups.pop("valid")
for grouper_str, grouper_kwargs in GROUPS:
    grouper_cls = import_object(grouper_str)
    ## We need this because lambda functions are not JSON-serializable
    if issubclass(grouper_cls, CallableGrouper):
        for k, v in list(grouper_kwargs.items()):
            if k == "function" and isinstance(v, str):
                grouper_kwargs[k] = eval(v)  # sorry :)
    ## End of lambda hack
    grouper = grouper_cls(**grouper_kwargs)        
    for group_key in list(groups.keys()):
        grouper.assign(groups[group_key], overwrite=True, progress=False)
        for subkey, subgroup in groups.pop(group_key).split_by_groups().items():
            groups[group_key + (subkey,)] = subgroup
print("10 groups to show keys:", *list(groups.keys())[:10], sep="\n")

10 groups to show keys:
('valid', 'AAK1', 'PercentageDisplacementMeasurement')
('valid', 'ABL1-nonphosphorylated', 'PercentageDisplacementMeasurement')
('valid', 'ABL2', 'PercentageDisplacementMeasurement')
('valid', 'ACVR1', 'PercentageDisplacementMeasurement')
('valid', 'ACVR1B', 'PercentageDisplacementMeasurement')
('valid', 'ACVR2A', 'PercentageDisplacementMeasurement')
('valid', 'ACVR2B', 'PercentageDisplacementMeasurement')
('valid', 'ACVRL1', 'PercentageDisplacementMeasurement')
('valid', 'ADCK3', 'PercentageDisplacementMeasurement')


## Write tensors to disk

Output files are written to `_output/<PIPELINE>/<DATASET>/<GROUP>.npz` files.

Each `npz` will contain two `np.ndarray` objects: `X` (featurized systems) and `y` (associated measurements), plus the train/test/validation indices.

In [13]:
random_grouper = RandomGrouper(TRAIN_TEST_VAL_KWARGS)

for group, ds in sorted(groups.items(), key=lambda kv: len(kv[1]), reverse=True):
    indices = random_grouper.indices(ds)
    X = np.asarray(ds.featurized_systems())
    y = ds.measurements_as_array()
    np.savez(OUT / f"{'__'.join([g for g in group if g != 'valid'])}.npz", X=X, y=y.astype("float32"), **indices)

# Reproducibility logs

In [14]:
%%capture cap --no-stderr
from kinoml.utils import watermark

w = watermark()

In [15]:
import json

with open(OUT/ "watermark.txt", "w") as f:
    f.write(cap.stdout)

with open(OUT / "hparams.json", "w") as f:
    json.dump(_hparams, f, default=str, indent=2)